<a href="https://colab.research.google.com/github/adityaghai07/mlflow_YOLO_Tracking/blob/main/MLFlow_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow ultralytics roboflow pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6

In [2]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [10]:
import os
import mlflow
import mlflow.pytorch
import roboflow
from ultralytics import YOLO
import yaml

# Ensure MLflow tracking is active
mlflow.set_tracking_uri("file:" + os.path.join(os.getcwd(), "mlruns"))

# Set up MLflow experiment
mlflow.set_experiment("exp1")

# Login to Roboflow and download dataset
roboflow.login()
rf = roboflow.Roboflow()
project = rf.workspace("model-examples").project("football-players-obj-detection")
dataset = project.version(2).download("yolov8")

# Explicitly create a new MLflow run
with mlflow.start_run(run_name="run2"):
    try:
        # Log dataset information
        with open(os.path.join(dataset.location, 'data.yaml'), 'r') as f:
            dataset_config = yaml.safe_load(f)

        mlflow.log_params({
            "dataset_name": "Football Players Object Detection",
            "dataset_version": 2,
            "total_classes": len(dataset_config['names']),
            "train_images": len(dataset_config['train']),
            "val_images": len(dataset_config['val'])
        })

        # Train the model
        model = YOLO('yolov8s.pt')
        results = model.train(
            data=dataset.location + '/data.yaml',
            epochs=1,
            imgsz=800,
            plots=True
        )

        # Log performance metrics
        metrics = {
            "mAP50": results.results_dict['metrics/mAP50(B)'],
            "mAP50-95": results.results_dict['metrics/mAP50-95(B)'],
            "precision": results.results_dict['metrics/precision(B)'],
            "recall": results.results_dict['metrics/recall(B)']
        }
        mlflow.log_metrics(metrics)

        # Log key artifacts
        artifacts = [
            f'{os.getcwd()}/runs/detect/train/confusion_matrix.png',
            f'{os.getcwd()}/runs/detect/train/results.png',
            f'{os.getcwd()}/runs/detect/train/labels.jpg',
            f'{os.getcwd()}/runs/detect/train/val_batch0_pred.jpg'
        ]

        for artifact in artifacts:
            if os.path.exists(artifact):
                mlflow.log_artifact(artifact)

# {HOME}/runs/detect/train/weights/best.pt
        # Log the trained model
        mlflow.log_artifact(f'{os.getcwd()}/runs/detect/train/weights/best.pt')

        # Optional: Log the model in MLflow's model format for easier deployment
        # mlflow.pytorch.log_model(model.model, "yolov8_model")

        print("Training complete. Check MLflow UI for experiment details.")

    except Exception as e:
        mlflow.set_tag("training_error", str(e))
        raise

You are already logged into Roboflow. To make a different login,run roboflow.login(force=True).
loading Roboflow workspace...
loading Roboflow project...
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/football-players-obj-detection-2/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms

train: Scanning /content/football-players-obj-detection-2/train/labels.cache... 298 images, 0 backgrounds, 0 corrupt: 100%|██████████| 298/298 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/football-players-obj-detection-2/valid/labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


...

MLflow: logging run_id(865ca60ca4de4661bcd88bcd7f8bdee0) to file:/content/mlruns
MLflow: disable with 'yolo settings mlflow=False'
TensorBoard: model graph visualization added ✅
Image sizes 800 train, 800 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      7.52G      1.612      2.532     0.9802        298        800: 100%|██████████| 19/19 [00:09<00:00,  1.95it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]


                   all         49       1174      0.964      0.223      0.284      0.161

1 epochs completed in 0.007 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 22.5MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.35it/s]


                   all         49       1174      0.963      0.224      0.284      0.161
                  ball         45         45          1          0    0.00242    0.00169
            goalkeeper         38         39          1          0      0.166     0.0757
                player         49        973       0.85      0.894      0.901      0.525
               referee         49        117          1          0      0.067      0.043
Speed: 0.3ms preprocess, 6.6ms inference, 0.0ms loss, 11.1ms postprocess per image
Results saved to runs/detect/train2
MLflow: results logged to file:/content/mlruns
MLflow: disable with 'yolo settings mlflow=False'
Training complete. Check MLflow UI for experiment details.


In [9]:
mlflow.end_run()

In [ ]:
!pip install mlflow --quiet

import mlflow
import os
from getpass import getpass

os.environ['MLFLOW_TRACKING_USERNAME'] = input('Enter your DAGsHub username: ')
os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('Enter your DAGsHub access token: ')
os.environ['MLFLOW_TRACKING_PROJECTNAME'] = input('Enter your DAGsHub project name: ')

mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME'] + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')

with mlflow.start_run(run_name="MLflow on Colab"):
  mlflow.log_metric("m1", 2.0)
  mlflow.log_param("p1", "mlflow-colab")

In [5]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [6]:
!pip install pyngrok

In [7]:

from pyngrok import ngrok
from getpass import getpass

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = getpass('2hee2LezWUSzOufYEMxLqKOwCZT_6aF6wi5z5JHRkbnD8peAe')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

2hee2LezWUSzOufYEMxLqKOwCZT_6aF6wi5z5JHRkbnD8peAe··········


Exception in thread Thread-31 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


MLflow Tracking UI: https://69c5-34-16-182-52.ngrok-free.app
